### 模型变大精度不一定更优
但只要保证变大后的模型包含小模型，那么大模型的性能是严格优于小模型的  
通过残差块 f(x) = g(x) + x 保证后面的层至少能学习到x

### ResNet的梯度计算
反向计算时：接近输出的梯度较大，而接近数据端的梯度由于连续的乘法会较小，导致学习到后期时前面的层很难再进行学习。  
ResNet通过加法保证了梯度随反向深度不会减小太多，前一层至少约有后一层的梯度大小，从而保证了近数据端的更新速度。

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [2]:
class Residual(nn.Module):
    def __init__(self, input_channels, num_channels, use_1x1conv=False, strides=1):
        # num_channels为输出channel数 
        super().__init__() 
        self.conv1 = nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels, kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
        self.relu = nn.ReLU(inplace=True) # inplace原地操作，不创建新变量，对原变量操作，节约内存

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)



In [5]:
# 输入和输出形状一致 的残差块
blk = Residual(3, 3) # 输入三通道，输出三通道
X = torch.rand(4, 3, 6, 6) 
Y = blk(X) # stride用的默认的1，所以宽高没有变化。如果strides用2，则宽高减半
print(Y.shape)
# 增加输出通道数的同时减半输出的高和宽 的残差块
blk = Residual(3, 6, use_1x1conv=True, strides=2)  # 由3变为6，通道数加倍
blk(X).shape

torch.Size([4, 3, 6, 6])


torch.Size([4, 6, 3, 3])

In [ ]:
# ResNet的第一个stage
b1 = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
    nn.BatchNorm2d(64), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

# class Residual为小block，resnet_block 为大block，#* 为Resnet网络的一个stage

def resnet_block(input_channels, num_channels, num_residuals, first_block=False):
    '''
    num_residuals：resnet_block中所含的Residual块数
    first_block：第一个Residual块默认减半，设为True则此stage块中第一个Residual块不减半
    '''
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block: 
            # 如果
            blk.append(Residual(input_channels, num_channels, use_1x1conv=True, strides=2))   
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

# 因为b1做了两次宽高减半，nn.Conv2d、nn.MaxPool2d，所以b2中的首次就不减半了  
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))     
# b3、b4、b5的首次卷积层都减半
b3 = nn.Sequential(*resnet_block(64, 128, 2)) 
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

net = nn.Sequential(
    b1, b2, b3, b4, b5,
    nn.AdaptiveAvgPool2d((1,1)),
    nn.Flatten(),
    nn.Linear(512,10))    

In [7]:
# 观察一下ReNet中不同模块的输入形状是如何变化的
X = torch.rand(size=(1,1,224,224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t',X.shape) # 通道数翻倍、模型减半

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 128, 28, 28])
Sequential output shape:	 torch.Size([1, 256, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])


In [ ]:
from my_model import my_train

In [ ]:
# 训练模型
# lr, num_epochs, batch_size = 0.05, 10, 256
lr, num_epochs, batch_size = 0.001, 10, 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)  
my_train(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu(), stratergy="Adam")